# ChatOllama Tool Calling Example: Simple Calculator

This notebook demonstrates how to define a simple calculator as a tool and let ChatOllama call it automatically.

In [24]:
from langchain.agents import Tool
from langchain_ollama import ChatOllama

# Python functions
def add(a, b): return a + b
def subtract(a, b): return a - b
def multiply(a, b): 
    print("--------------->")
    return a * b
def divide(a, b): return a / b if b != 0 else "Error: division by zero"

# Tools
tools = [
    Tool(name="Add", func=add, description="Adds two numbers a and b"),
    Tool(name="Subtract", func=subtract, description="Subtracts b from a"),
    Tool(name="Multiply", func=multiply, description="Multiplies a and b"),
    Tool(name="Divide", func=divide, description="Divides a by b")
]

# LLM
llm = ChatOllama(
    model="deepseek-r1:7b",
    
    )

# Agent
from langchain.agents import initialize_agent, AgentType
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [ ]:
# Initialize ChatOllama
chat = ChatOllama(model="deepseek-r1:8b")  # replace with your model

# Test a simple tool call
response = chat.invoke("Calculate 45 multiplied by 67 using the Multiply tool")



> Entering new None chain...


CancelledError: 

In [13]:
# Inspect the tool calls
response.tool_calls

AttributeError: 'dict' object has no attribute 'tool_calls'

In [5]:
# Example: Extract the tool arguments and calculate the result manually
if response.tool_calls:
    tool_call = response.tool_calls[0]
    args = tool_call['args']
    result = args['a'] * args['b']
    print(f"Tool called: {tool_call['name']}")
    print(f"Arguments: a={args['a']}, b={args['b']}")
    print(f"Result: {result}")

In [ ]:
from langchain.agents import Tool, initialize_agent, AgentType
from langchain_ollama import ChatOllama

# Tools
async def multiply_async(a, b):
    import asyncio
    await asyncio.sleep(0.1)  # simulate async work
    return a * b

tools = [
    Tool(name="Multiply", func=multiply_async, description="Multiplies two numbers")
]

# LLM
llm = ChatOllama(model="deepseek-r1:7b")

# Agent
agent = initialize_agent(
    
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Stream
response_text = ""
async for chunk in agent.astream("Calculate 45 * 67 using Multiply tool"):
    if chunk.content:
        response_text += chunk.content
        print(chunk.content, end="")  # streaming output




> Entering new None chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `<think>
To calculate \(45 \times 67\) using multiplication, I'll break it down into smaller parts for easier computation.

First, multiply 45 by 60:
\(45 \times 60 = 2700\).

Next, multiply 45 by 7:
\(45 \times 7 = 315\).

Finally, add the two results together to get the total product:
\(2700 + 315 = 3015\).
</think>

To calculate \(45 \times 67\) using the Multiply tool:

**Step 1:**  
Multiply 45 by 60.
\[ 45 \times 60 = 2700 \]

**Step 2:**  
Multiply 45 by 7.
\[ 45 \times 7 = 315 \]

**Step 3:**  
Add the results from Step 1 and Step 2.
\[ 2700 + 315 = 3015 \]

\[
\boxed{3015}
\]`
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

### How It Works
- We define a tool as a **Pydantic model** (`Multiply`) with fields for the inputs.
- ChatOllama can **call tools automatically** if the prompt indicates the tool usage.
- The `tool_calls` field in the response contains the tool name and arguments.
- You can then execute the tool logic in Python or elsewhere.

### Next Steps
- Add more operations like Add, Subtract, Divide as separate tools.
- Combine with **streaming** to see partial tool call results.
- Use JSON output format if you want structured responses for frontend consumption.

In [32]:
from langchain.agents import AgentExecutor
from langchain.llms.base import LLM
from langchain.agents import Tool

# Example tools
def add(a, b): return a + b
def multiply(a, b): return a * b

tools = [
    Tool(name="Add", func=add, description="Add two numbers"),
    Tool(name="Multiply", func=multiply, description="Multiply two numbers")
]

# Your LLM
from langchain_ollama import ChatOllama
llm = ChatOllama(model="deepseek-r1:7b")

# Create an agent manually
from langchain.agents.react.base import ReACTAgent
agent = ReACTAgent.from_llm_and_tools(llm=llm, tools=tools)

# Wrap in AgentExecutor
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Run
result = executor.run("Calculate 45 * 67 using Multiply tool")
print(result)


ImportError: cannot import name 'ReACTAgent' from 'langchain.agents.react.base' (d:\WevN\WevN_ofiicial_frontend\WevN\backend\server\venv\Lib\site-packages\langchain\agents\react\base.py)

In [36]:
import asyncio
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

# IMPORTANT: Use the new import path
from langchain_ollama import ChatOllama 

# 1. Define your tools
@tool
def get_weather(city: str) -> str:
    """Gets the current weather for a specified city."""
    if city.lower() == "new york":
        return "It's 24°C and sunny in New York. ☀️"
    elif city.lower() == "london":
        return "It's 15°C and cloudy in London. ☁️"
    else:
        return f"I don't have the weather for {city}."

# 2. Initialize the model and bind the tools
# Make sure your Ollama server is running with a model that supports tool calling
# The model name here is an example; use the one you have, e.g., "llama3"
llm = ChatOllama(model="deepseek-r1:7b", temperature=0)
tools = [get_weather]
llm_with_tools = llm.bind_tools(tools) # This will now work correctly

# The rest of your async main function remains the same...
async def main():
    messages = [HumanMessage(content="What's the weather like in New York?")]
    
    print("--- Streaming tool call ---")
    async for chunk in llm_with_tools.astream(messages):
        # The output chunk will contain the tool_calls attribute
        if chunk.tool_calls:
            print(chunk.tool_calls)
    
    # You would continue here with the logic to execute the tool
    # and stream the result back to the model.

await main()

--- Streaming tool call ---


C:\Program Files\Python313\Lib\dis.py:205: RuntimeWarning: coroutine 'main' was never awaited
  def _deoptop(op):


ResponseError: registry.ollama.ai/library/deepseek-r1:7b does not support tools (status code: 400)

In [ ]:
import asyncio
# Imports are slightly changed
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool # <-- Import the decorator

# Add the @tool decorator above each function
@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers a and b."""
    print(f"--- TOOL: Executing multiply({a}, {b}) ---")
    return a * b

@tool
def add(a: int, b: int) -> int:
    """Adds two numbers a and b."""
    print(f"--- TOOL: Executing add({a}, {b}) ---")
    return a + b

model="phi3:mini"
# model = "llama3.1:8b"
    
# 1. Choose a tool-calling model
llm = ChatOllama(model= model, temperature=0)

# The tools list is now simpler
tools = [add, multiply]

# 2. Create the Agent using a modern prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that is good at math."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt)

# 3. Create the Agent Executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)


async def stream_agent_response():
    """
    Streams the agent's response, including intermediate steps.
    """
    async for event in agent_executor.astream_events(
        {"input": "What is 3 plus 5, and then multiply the result by 10?"},
        version="v2"
    ):
        event_type = event["event"]
        
        if event_type == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if chunk.content:
                print(chunk.content, end="", flush=True)

        elif event_type == "on_tool_start":
            print(f"\n--- TOOL START: {event['name']} with args {event['data'].get('input')} ---")
            
        elif event_type == "on_tool_end":
            print(f"\n--- TOOL END: Tool {event['name']} output was: {event['data'].get('output')} ---")

# In Jupyter, you can just 'await' the async function
await stream_agent_response()

ResponseError: registry.ollama.ai/library/phi3:mini does not support tools (status code: 400)

In [45]:
import asyncio
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers a and b."""
    print(f"--- TOOL: Executing multiply({a}, {b}) ---")
    return a * b

@tool
def add(a: int, b: int) -> int:
    """Adds two numbers a and b."""
    print(f"--- TOOL: Executing add({a}, {b}) ---")
    return a + b
    
# Use the phi3:mini model you pulled
llm = ChatOllama(model="qwen3:4b", temperature=0)

tools = [add, multiply]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that is good at math."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

async def stream_agent_response():
    async for event in agent_executor.astream_events(
        {"input": "What is 3 plus 5, and then multiply the result by 10?"},
        version="v2"
    ):
        # ... (rest of the streaming logic)
        event_type = event["event"]
        if event_type == "on_chat_model_stream":
            chunk = event["data"]["chunk"]
            if chunk.content:
                print(chunk.content, end="", flush=True)
        elif event_type == "on_tool_start":
            print(f"\n--- TOOL START: {event['name']} with args {event['data'].get('input')} ---")
        elif event_type == "on_tool_end":
            print(f"\n--- TOOL END: Tool {event['name']} output was: {event['data'].get('output')} ---")

# In Jupyter, you can just 'await' the async function
await stream_agent_response()

<think>
Okay, let's see. The user is asking for 3 plus 5, and then multiply the result by 10. Hmm, so first I need to do 3 + 5. That's straightforward. Then take that result and multiply by 10.

Wait, but the tools provided are add and multiply functions. So I need to call these functions step by step. Let me think.

First step: add 3 and 5. So the add function takes a and b as integers. So I should call add with a=3 and b=5. That gives me 8. Then, take that result (8) and multiply by 10. So the multiply function would take a=8 and b=10. But wait, the user says "multiply the result by 10", so the second step is multiply the sum by 10.

But the problem is, the user wants the final answer, but the tools are functions that I can call. However, the assistant is supposed to return the tool calls. Wait, the user is asking for the answer, but the instructions say that the assistant may call one or more functions to assist with the user query.

Wait, the user's question is a math problem, so t

CancelledError: 